<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EC%97%AD%EC%A0%84%ED%8C%8C%EB%A5%BC_%ED%86%B5%ED%95%9C_%ED%95%99%EC%8A%B5_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from collections import OrderedDict

# 20분

# 필요한 함수와 클래스

In [ ]:
def _numerical_gradient_no_batch(f, x): # 수치 gradient, 앞의 수치미분과 같은 개념
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성(이 경우 x와 같은 차원의 제로 벡터)

    # gradient는 각 축에 대한 방향 미분을 모아놓은 벡터이므로 각 방향에 대한 수치미분을 grad 벡터에 차례로 대입하는 과정이다.
    for idx in range(x.size):
        tmp_val = x[idx]

        # f(x+h) 계산
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)

        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] =  (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원

    return grad

In [ ]:
def numerical_gradient(f, X): # 배치처리를 한 경우(행렬, 텐서 미분)
    if X.ndim == 1: # 1차원, 그러니까 행렬로 묶이지 않은 벡터인 경우
        return _numerical_gradient_no_batch(f, X)
    else:
        grad = np.zeros_like(X)

        for idx, x in enumerate(X): # enumerate 설명1, 행벡터를 차례로 불러와 미분한다.
            grad[idx] = _numerical_gradient_no_batch(f, x)

        return grad

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # y가 벡터인 경우, 배치처리를 안 한 경우
        t = t.reshape(1, t.size) # 10차원 벡터를 행렬로 변환.(개념상.)
        y = y.reshape(1, y.size)

    if t.size == y.size: # 라벨이 원-핫 인코딩이 돼어 있다면, 그러니까 입력 값이 10차원 벡터라면, size함수는 메트릭스의 원소갯수를 반환한다.
        t = t.argmax(axis=1) # 원-핫 인코딩 이전으로 되돌리기 위해 작성.

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size # 설명1, 확률벡터로 이루어진 행렬에서 알맞은 인덱스를 얻기위한 과정.

In [ ]:
def softmax(x):
    if x.ndim == 2:   # x가 행렬일 경우, 즉 배치처리를 해서 각 행이 확률벡터인 메트릭스를 입력받은 경우.
        x = x.T # 각 행이 확률벡터인 초기 메트릭스에서 각 열이 확률벡터인 메트릭스로 전치 시킨다.
        x = x - np.max(x, axis = 0) # np.max(x, axis = 0)는 각 열에 대한 최댓값을 저장한 리스트(벡터)를 반환 한다. 그리고 x에 대한 - 연산은 각 열의 확률벡터에 각 열의 최댓값을 빼주는 연산이다.(밑의 오버플로우 방지를 위한 최댓값 빼기와 같음.)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0) # 위와 같은 행렬과 벡터의 연산 원리.( 밑의 예제 참고.)
        return y.T

    x = x - np.max(x) # 오버플로를 방지하기위해 작성. 노트참고. , vector에 scalar를 빼면 numpy에서 알아서 원소별로 연산을 한다.
    return np.exp(x) / np.sum(np.exp(x))  # normalize

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # np.exp(x)는 밑이 자연상수 e 이고 x가 지수인 수를 반환한다.

In [ ]:
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

In [ ]:
# 4차원 텐서를 염두해둔 코드이기 떄문에 다음 학기때완전히 이해가능
class Affine:
    def __init__(self, W, b):
        self.W = W # 가중치 행렬
        self.b = b # bias 벡터

        # 입력된 data 초기화
        self.x = None
        self.original_x_shape = None
        # 가중치와 bias 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1) # 4차원 텐서를 행렬로 변환. -1을 입력하면 reshape 가능한 숫자를 알아서 대입한다.
        self.x = x

        out = np.dot(self.x, self.W) + self.b # affine 변환. bias의 경우 배치처리를 할경우 행렬뎃섬에서 shape이 맞지 않는데 numpy에서 자동으로 벡터를 복사해 각 열에 더하게 된다.

        return out

    def backward(self, dout): # dout:흘러들어온 미분
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0) # 행렬의 경우 axis 0: 행, 1: 열. 텐서의 경우 axis 0: 각 행렬, 1: 각 행렬의 행, 2: 각 행렬의 열

        dx =dx.reshape(*self.original_x_shape) # 입력데이터

        return dx

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0) # self.mask는 0보다 크면 false 0보다 작으면 true인 리스트를 반환한다.
        out = x.copy()
        out[self.mask] = 0 # 설명1

        return out

    def backward(self, dout):
        dout[self.mask] = 0 # 위의 설명1을 참조. 주의할점은 흘러들어온 미분값의 부호에 의해 1이나 0을 곱하는것이 정해지는게 아니라 위의 x값을 기준으로 gradient를 살리거나 죽이게 된다.
        dx = dout

        return dx

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실함수
        self.y = None # softmax의 출력(확률 벡터)
        self.t = None # 정답 레이블(one-hot encoding 형태)

    def forward(self, x,  t):
        self.t = t
        self.y = softmax(x)
        self.loss = corss_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout = 1): # 역전파의 시작은 1 이다.
        batch_size = self.t.shape[0]

        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩이 돼어 있는 경우.
            dx = (self.y - self.t) / batch_size # 손실함수의 정의 값은 각각의 손실함수값의 평균이므로 배치사이즈로 나누어 준다.

        else: # 원-핫 이코딩이 돼어 있지 않은 경우.
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1 # 도출된 확률벡터는 배치처리로 묶여있다는 것을 생각해보면 위와 같은 결과를 도출함.
            dx = dx / batch_size

            return dx

# TwoLayerNet

In [ ]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01): # 파라미터들을 초기화한다. 편향은 0, 가중치는 지정한 정규분포에서 랜덤하게 초기화 한다.
                                                                                      # std는 표준편차이고 특별히 지정하지 않으면 0.01로 지정한다.

        # 가중치 초기화
        self.params = {} # 빈 딕셔너리.

        # 입력층에서 은닉층으로의 아파인 변환에 필요한 가중치와 편향
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) # 표준정규분포(평균이 0이고 표준편차가 1)에 상수 a를 곱하면 평균이 0이고 표준편차가 a인 정규분포를 생성한다.
        self.params['b1'] = np.zeros(hidden_size) # bias는 0으로 지정한다.

        # 은닉층에서 출력층으로의 아파인 변환에 필요한 가중치와 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict() # OrderedDict()은 순서가 있는 딕셔너리를 만들어 준다.(원래 딕셔너리는 순서가 없음.) 파이썬 3.7버전 이상부터는 ordered 딕셔너리를 사용할 필요가 없다.
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1']) # Affine 클래스의 인스턴스를 생성해서 딕셔너리에 추가
        self.layers['Relu1'] = Relu() # Relu 클래스의 인스턴스를 생성해서 딕셔너리에 추가
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2']) # Affine 클래스의 인스턴스를 생성해서 딕셔너리에 추가

        self.lastLayers = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values(): # 딕셔너리의 value값(각 층의 인스턴스)를 차례로 대입
            x = layer.forward(x)

        # predict 함수의 결과는 score로 softmax함수로 확률벡터로 변환되기전의 값이 도출된다.
        # 모델의 학습이 끝나고 추론할 때에는 softmax층과 loss층의 연산은 계산비용의 낭비일 뿐 이다. 더 이상 학습이 필요없고 최대 전수가 나오는 항만 알면 되기 때문이다.
        return x

    def loss(self, x, t): # 추론한 확률분포와 라벨의 확률분호의 거리를 측정 한다.(cross entropy)
        y = self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1) # 행을 따라 최댓값의 인덱스를 리스트로 반환한다.
        t = np.argmax(t, axis = 1)

        accuracy = np.sum(y == t) / float(x.shape[0]) # 정답의 수를 입력한 데이터 수로 나눈다.(배치 처리)

        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {} # 빈 딕셔너리
        grads['W1'] = numerical_gradient(loss_W, self.params['W1']) # 785*50 행렬 반환 (gardient의 shape은 입력 shpae과 동일한다.)
        grads['b1'] = numerical_gradient(loss_W, self.params['b1']) # 50차원 백터 반환
        grads['W2'] = numerical_gradient(loss_W, self.params['W2']) # 50*10 행렬 반환
        grads['b2'] = numerical_gradient(loss_W, self.params['b2']) # 10차원 벡터 반환

        return grads

    def gradient(self, x, t): # 역전파를 구현한 코드. 수치미분은 계산비용이 너무 크기 때문 모델을 학습시키기에는 부적합하다.
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}

        batch_num = x.shape[0]

        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        #backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis = 0)

        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis = 0)

        return grads